In [ ]:
!pip install keras-tuner

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Conv1D, GlobalMaxPooling1D, Flatten, MaxPooling1D
from tensorflow.keras.layers import Bidirectional

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.optimizers import SGD, Adam

import time
import keras_tuner as kt

In [ ]:
# load the dataset from static csv
# df = pd.read_csv('data/PFG.csv', header=0, index_col=0, parse_dates=True, usecols=['Date','Adj Close'])
# df.head(2)

### Load Data using Pandas Datareader

In [ ]:
!pip install pandas-datareader

In [ ]:
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
import pandas_datareader.data as pdr
import yfinance as yf

yf.pdr_override()
df = pdr.get_data_yahoo('PFG')#, start, end)

df = df[['Adj Close']]  
df.head()

In [ ]:
df.tail()

In [ ]:
dataset = df.values.astype('float32')
dataset

In [ ]:
dataset.shape

In [ ]:
split_pct = 0.8

split = int(split_pct*len(dataset))

train = dataset[:split]
valid = dataset[split:]

print(len(train))
print(len(valid))

train.shape, valid.shape

In [ ]:
# skip this cell if you don't want scaling

#scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler = StandardScaler(with_mean=True, with_std=True)
train = scaler.fit_transform(train)
valid = scaler.transform(valid)

train.shape, valid.shape

#### Generate Timeseries Sequences

https://keras.io/api/preprocessing/timeseries/#timeseriesgenerator-class

In [ ]:
lags = 30

train_generator = TimeseriesGenerator(train, train, length=lags, batch_size=5)
valid_generator = TimeseriesGenerator(valid, valid, length=lags, batch_size=1)

In [ ]:
train_generator[0][1]

### Build Models

In [ ]:
def cnnlstm_model_builder(hp):
    model = Sequential()
    model.add(Conv1D(filters=hp.Choice('conv1_filters', values=[32,64]),
                     kernel_size=hp.Choice('conv1_size', values=[1,3]),
                     activation='relu', input_shape=(lags, 1)))
    model.add(MaxPooling1D())
    model.add(LSTM(units=hp.Choice('lstm1_units', values=[64,128,256]),
                   dropout=hp.Float('lstm1_do_rate', min_value=0, max_value=.3, sampling='linear')))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])))
    return model

In [ ]:
# for reproducibility
SEED = 1

# number of hyperparameter combinations (number of rounds) that will be tested by the tuner
MAX_TRIALS = 10

# number of models that should be built and fit for each trial for robustness purposes
EXECUTION_PER_TRIAL = 2

BAYESIAN_NUM_INITIAL_POINTS = 1

# directory for each search
RANDOM_DIR = "random-search-{}".format(int(time.time()))
BAYESIAN_DIR = "bayesian-search-{}".format(int(time.time()))

In [ ]:
random_tuner = kt.RandomSearch(cnnlstm_model_builder, objective='val_loss', seed=SEED, max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL, directory=RANDOM_DIR, project_name='ts_cnnlstm_tuner')

bayesian_tuner = kt.BayesianOptimization(cnnlstm_model_builder, objective='val_loss', num_initial_points=BAYESIAN_NUM_INITIAL_POINTS, seed=SEED, max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL, directory=BAYESIAN_DIR, project_name='ts_cnnlstm_tuner')

In [ ]:
random_tuner.search(train_generator, epochs=20, verbose=1, validation_data=valid_generator)

In [ ]:
bayesian_tuner.search(train_generator, epochs=20, verbose=1, validation_data=valid_generator)